In [1]:
!zipline bundles

Korea_commodity 2023-12-05 21:01:08.694331
Korea_commodity 2023-12-05 09:01:34.202227
csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>


In [2]:
!zipline ingest -b Korea_commodity

[2023-12-06T03:01:51+0600-INFO][zipline.data.bundles.core]
 Ingesting Korea_commodity
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]   


In [3]:
from zipline.api import order_target, record, symbol


def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)

In [9]:
import pandas as pd
from zipline.utils.calendar_utils import get_calendar
from zipline import run_algorithm
calendar = get_calendar('XKRX')

In [10]:
start_date = pd.to_datetime('2022-11-22 00:00:00.000000', format='%Y-%m-%d %H:%M:%S.%f')
end_date = pd.to_datetime('2023-05-02 00:00:00.000000', format='%Y-%m-%d %H:%M:%S.%f')

In [11]:
results = run_algorithm(start=start_date,
                         end=end_date,
                         initialize=initialize,
                         handle_data=handle_data,
                         capital_base=10000,
                         benchmark_returns=None,
                         trading_calendar=calendar,
                         bundle='Korea_commodity',
                         data_frequency='daily',
                         )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/popeye/.local/lib/python3.10/site-packages/zipline/assets/assets.py", line 783, in _lookup_symbol_strict
    owners = ownership_map[company_symbol, share_class_symbol]
KeyError: ('AAPL', '')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/popeye/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4940/1196516867.py", line 1, in <module>
    results = run_algorithm(start=start_date,
  File "/home/popeye/.local/lib/python3.10/site-packages/zipline/utils/run_algo.py", line 397, in run_algorithm
    return _run(
  File "/home/popeye/.local/lib/python3.10/site-packages/zipline/utils/run_algo.py", line 227, in _run
    ).run()
  File "/home/popeye/.local/lib/python3.10/site-packages/zipline/algorithm.py", line 625, in run
    for perf in self.get_generator